In [ ]:
#########################################
## granite-3.3-2b-instruct en Colab
#########################################

In [ ]:
# installations

import requests, os, re, torch, ast, time

from transformers import AutoTokenizer, AutoModelForCausalLM

from transformers.utils import get_json_schema

from datetime import datetime, timezone, timedelta


In [ ]:
TOKENIZER = AutoTokenizer.from_pretrained("ibm-granite/granite-3.3-2b-instruct")

GOOGLE_API_KEY = os.getenv('GOOGLE API KEY',"xxxxxxxxxxxxxxx")

model_path="ibm-granite/granite-3.3-2b-instruct"

device="cuda"

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [ ]:
def get_geolocation() :
  return "Avenida Complutense 30, Madrid, España"

def geocode(address):
    if address == 'None':
      address = get_geolocation()
    geocode_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={GOOGLE_API_KEY}"
    response = requests.get(geocode_url)
    results = response.json()["results"]
    if not results:
        raise ValueError(f"No se encontró la dirección: {address}")
    location = results[0]["geometry"]["location"]
    return location["lat"], location["lng"]


def get_route_duration(origin: 'None', end: 'None') -> dict:

    """
    Devuelve la duración del trayecto en automóvil desde una dirección de origen a una de destino, y un valor que indica si hay atasco
    en el trayecto.

    Args:

    origin: el punto de comienzo, en formato cadena de caracteres, por ejemplo, "calle de Torres Quevedo 189, Madrid, España"

    end: destino del trayecto, en formato cadena de caracteres.

    Returns:

    Un diccionario con la respuesta de duración del trayecto y un indicador sobre si hay atasco en la ruta.

    """

    print(f"Calculando distancia y duración desde {origin} a {end}")

    try:

      lat_origen, lon_origen = geocode(origin)
      lat_destino, lon_destino = geocode(end)
      departure_dt = datetime.now(timezone.utc) + timedelta(minutes=1)
      departure_iso_string = departure_dt.strftime('%Y-%m-%dT%H:%M:%SZ')

      route_url = "https://routes.googleapis.com/directions/v2:computeRoutes"
      headers = {
       "Content-Type": "application/json",
       "X-Goog-Api-Key": GOOGLE_API_KEY,
        "X-Goog-FieldMask": "*"
      }
      # Primero calculamos la duración del trayecto sin tráfico
      route_data = {
        "origin": {
            "location": {
                "latLng": {"latitude": lat_origen, "longitude": lon_origen}
            }
        },
        "destination": {
          "location": {
              "latLng": {"latitude": lat_destino, "longitude": lon_destino}
          }
        },
        "travelMode": "DRIVE",
        "routingPreference": "TRAFFIC_UNAWARE",
        "computeAlternativeRoutes": False,
        "routeModifiers": {
            "avoidTolls": True,
            "avoidHighways": False,
            "avoidFerries": True
        },
        "languageCode": "es-ES",
        "units": "METRIC"
      }
      route_response = requests.post(route_url, headers=headers, json=route_data)
      route_duration_non_traffic = int(route_response.json()["routes"][0]["duration"].rstrip("s"))

      # Calculamos el tiempo del trayecto con tráfico

      route_data = {
        "origin": {
            "location": {
                "latLng": {"latitude": lat_origen, "longitude": lon_origen}
            }
        },
        "destination": {
          "location": {
              "latLng": {"latitude": lat_destino, "longitude": lon_destino}
          }
        },
        "travelMode": "DRIVE",
        "routingPreference": "TRAFFIC_AWARE",
        "departureTime": departure_iso_string,
        "computeAlternativeRoutes": False,
        "routeModifiers": {
            "avoidTolls": True,
            "avoidHighways": False,
            "avoidFerries": True
        },
        "languageCode": "es-ES",
        "units": "METRIC"
      }
      route_response = requests.post(route_url, headers=headers, json=route_data)
      route_duration_traffic = int(route_response.json()["routes"][0]["duration"].rstrip("s"))

      print("segundos tráfico", route_duration_traffic)
      print("segundos non tráfico", route_duration_non_traffic)
      if route_duration_traffic >= route_duration_non_traffic*1.1 :
        traffic_jam = 'yes' # añadimos un item más al diccionario
      else :
        traffic_jam = 'no'

      return {

            "duration": round(route_duration_traffic/60), # damos la duración en minutos

            "traffic_jam": traffic_jam
      }

    except Exception as e:

        print(f"Error al buscar información en Google Maps: {e}")

        return {

            "duration": "none",

            "traffic_jam": "none"
        }


In [ ]:
from transformers.utils import get_json_schema
tools = [get_json_schema(get_route_duration)]
tools

In [ ]:
def make_api_request(instructions) -> str:

    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map=device,
        torch_dtype=torch.bfloat16,
    )

    model_parameters = {

        "do_sample": False, # es equivalente a "decoding_method": "greedy". Me da como salida el token más probable.

        "max_new_tokens": 2000,

        "repetition_penalty": 1.05,

        "eos_token_id" : TOKENIZER.eos_token_id # es equivalente a "stop_sequences": [TOKENIZER.eos_token] y sirve para indicarle al modelo
                                                # cuándo parar
    }

    output = model.generate(
        **instructions,
        **model_parameters,
    )

    tokenizer = AutoTokenizer.from_pretrained(model_path)

    prediction = tokenizer.decode(output[0], skip_special_tokens=False)
    print("prediction", prediction)
    print("***********************************************************************")
    prediction = tokenizer.decode(output[0, instructions["input_ids"].shape[1]:], skip_special_tokens=True)

    return prediction

In [ ]:
question= "¿Cómo está el tráfico para llegar a la Plaza de España en Madrid?"
#question= "¿Hay atasco para ir a la Plaza de España en Madrid?"
#question= "¿Cómo de densa está la circulación para ir a la Plaza de España de Madrid?"
query = question+" Teniendo en cuenta que el origen es "+ get_geolocation()

conversation = [{"role": "system","content": """Eres un asistente con acceso a las siguientes llamadas a función.
Tu tarea es producir una secuencia de llamadas a función en respuesta a la petición del usuario.
Muestra la respuesta en lenguaje natural, en español.
En tu resultado escribe los argumentos de la función tras "arguments" y el nombre de la función tras "function"
"""},

{"role": "user", "content": query},
]

instructions = TOKENIZER.apply_chat_template(conversation=conversation, tools=tools, tokenize=False, add_generation_prompt=False)
instructions

In [ ]:
instructions = TOKENIZER.apply_chat_template(conversation=conversation, tools=tools, return_tensors="pt", return_dict=True, add_generation_prompt=True).to(device)
instructions

In [ ]:
data_traffic = make_api_request(instructions)

data_traffic



In [ ]:
def tool_call(llm_response: str):

    tool_request = ast.literal_eval(re.search("({.+})", llm_response, re.DOTALL).group(0)) #re.DOTALL detecta todos los caracteres incluidos los saltos de linea
    print(tool_request)
    tool_name = tool_request["name"]

    tool_arguments = tool_request["arguments"]

    tool_response = globals()[tool_name](**tool_arguments)

    return tool_response


In [ ]:
tool_response_traffic = tool_call(data_traffic)

tool_response_traffic


In [ ]:
conversation2 =  [

    {"role": "system", "content":
    """
    Muestra la respuesta de la función externa en lenguaje natural, en español.
    """},

    {"role": "user", "content":
    """Haz que la respuesta sea muy breve.
    Tu respuesta debe ser muy corta, debes decir: "No hay atasco" si traffic_jam es 'no'. E incluir también en la respuesta la duración de trayecto
     en minutos.
    Tu respuesta debe ser muy corta, debes decir: "Hay atasco" si traffic_jam es 'yes'. E incluir también en la respuesta la duración de trayecto en minutos.
    No incluyas nombres de calles en tu respuesta
    No incluyas nombres de ciudades o pueblos en tu respuesta
    No incluyas nombres de países en tu respuesta
    """ },

    {"role": "tool_response", "content": str(tool_response_traffic) },

]

instruction_2 = TOKENIZER.apply_chat_template(conversation=conversation2, tokenize=False, add_generation_prompt=False)
instruction_2


In [ ]:
instructions_2 = TOKENIZER.apply_chat_template(conversation=conversation2, return_tensors="pt", return_dict=True, add_generation_prompt=True).to(device)

data_2 = make_api_request(instructions_2)

data_2


In [ ]:
conversation3 =  [

    {"role": "system", "content":
    """
    Muestra la respuesta de la función externa en lenguaje natural, en español.
    """},

    {"role": "user", "content":
    """Haz que la respuesta sea muy breve.
    Tu respuesta debe ser muy corta, debes decir: "No hay atasco" si traffic_jam es 'no'.
    Tu respuesta debe ser muy corta, debes decir: "Hay atasco" si traffic_jam es 'yes'.
    Responde en 3 palabras
    La respuesta debería ser: "No hay atasco" o "Hay atasco"
    No incluyas nombres de calles en tu respuesta
    No incluyas nombres de ciudades o pueblos en tu respuesta
    No incluyas nombres de países en tu respuesta
    """ },

    {"role": "tool_response", "content": str(tool_response_traffic) },

]

instruction_3 = TOKENIZER.apply_chat_template(conversation=conversation3, tokenize=False, add_generation_prompt=False)
instruction_3


In [ ]:
instructions_3 = TOKENIZER.apply_chat_template(conversation=conversation3, return_tensors="pt", return_dict=True, add_generation_prompt=True).to(device)
data_3 = make_api_request(instructions_3)

data_3